In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.python.keras.datasets import boston_housing
from tensorflow.contrib.eager.python import tfe

D:\Users\Yue\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# enable eager mode
tf.enable_eager_execution()
tf.set_random_seed(0)
np.random.seed(0)

In [3]:
if not os.path.exists('weights/'):
    os.makedirs('weights/')

# constants
batch_size = 128
epochs = 24

In [4]:
# dataset loading
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

# normalization of dataset
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)

x_train = (x_train - mean) / (std + 1e-8)
x_test = (x_test - mean) / (std + 1e-8)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print('x train', x_train.shape, x_train.mean(), x_train.std())
print('y train', y_train.shape, y_train.mean(), y_train.std())
print('x test', x_test.shape, x_test.mean(), x_test.std())
print('y test', y_test.shape, y_test.mean(), y_test.std())

x train (404, 13) 3.6316616e-10 1.0
y train (404,) 22.395049504950492 9.199035423364862
x test (102, 13) 0.02082699 0.98360837
y test (102,) 23.07843137254902 9.123806690181466


# Custom Layers

There is currently no way to add custom weight matrices or custom variables to a `tf.keras.Model`. However, we *can* quitely easily work around that by simply extending a **tf.keras.layers.Layer** !

A Custom Layer is in essence a Layer that we can treat as if it was a part of the Keras API itself. This includes the ability to build the model on demand, and register any layer automatically without having to resort to the messy hack that we had to do in `10_01_custom_models.ipynb`. 

There are 3 important functions (and 1 optional function) which must be overridden to write a custom layer : 

- `__init__` : Must pass **kwargs to its super constructior to maintain Keras API conventions


- `build(self, _)` : This must be extended, and the `_` must be replaced by the input_shape like I have done here. Use this input shape to build the variables of the model. 
    - Note : **It is important to set `self.built = True` at the end of build(). Otherwise, multiple copies of the weights with the same name will be built at each call of the layer, which will cause wrong training and the model wont be Checkpointable.**.
    - `self.built = True` can be automatically be set by calling the base method `super().build()`. It doesnt need to be passed the input shape, but if it is passed, it will be ignored and simply set `self.built = True` for us.


- `call(self, inputs, **kwargs)` : The same as models, but here, trainable and mask parameters come under **kwargs. They can be set as dictionary values in the call method. 
    - Use the variables and weights you build in side `build()` to perform the forward pass here.


- `compute_output_shape(self, input_shape)` : Optional override. Allows you to define the output shape of the layer. While this is not needed for Eager execution mode, it is **mandatory** for normal execution mode and ordinary Keras layers. Takes in the input_shape as a tuple and passes a tuple of integer shapes as its outputs.


In [5]:
# A "Custom" layer which mimics the Dense layer from Keras
class CustomLayer(tf.keras.layers.Layer):

    def __init__(self, dim, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.dim = dim

    # change the "_" for the input shape to some variable name, and build on first call !
    def build(self, input_shape):
        # add variable / add_weights works inside Layers but not Models !
        self.kernel = self.add_variable('kernel',
                                        shape=[input_shape[-1], self.dim],
                                        dtype=tf.float32,
                                        initializer=tf.keras.initializers.he_normal())

        # Do NOT forget to call this line, otherwise multiple model variables will be built with the same name
        # This cannot happen inside Keras layers, and therefore the model will not be Checkpointeable.
        # It also wont train properly.
        #
        self.built = True

    def call(self, inputs, **kwargs):
        return tf.matmul(inputs, self.kernel)

# Using a Custom Layer

Now that the layer has been written, use it just like any other custom layer written in Keras. It is now an extention to the Keras API, so all the same rules apply to the custom layer.

In [6]:
# model definition
class CustomRegressor(tf.keras.Model):
    def __init__(self):
        super(CustomRegressor, self).__init__()
        # self.add_variable and self.add_weight are not yet supported inside a Model
        # However, since we created a custom layer (Dense layer), we *can* attach it to this model
        # just like other layers !
        self.hidden1 = CustomLayer(1)

        # we also use a keras layer along with a custom weight matrix
        self.hidden2 = tf.keras.layers.Dense(1)

    def call(self, inputs, training=None, mask=None):
        output1 = self.hidden1(inputs)
        output1 = tf.keras.activations.relu(output1)

        output2 = self.hidden2(inputs)

        output = output1 + output2  # goofy model ; just for demonstration purposes
        return output

# Benefit

With the inbuilt support for all Keras layers, a Keras model can now use *all of the additional functionality* such as Model.fit() and Model.evaluate().

In [7]:
device = '/cpu:0' if tfe.num_gpus() == 0 else '/gpu:0'

with tf.device(device):
    # build model and optimizer
    model = CustomRegressor()
    model.compile(optimizer=tf.train.AdamOptimizer(1.), loss='mse')

    # suggested fix for TF <= 1.9; can be incorporated inside `_eager_set_inputs` or `_set_input`
    # Fix = Use exactly one sample from the provided input dataset to determine input/output shape/s for the model
    dummy_x = tf.zeros((1, 13))
    model.call(dummy_x)

    # Now that we have a "proper" Keras layer, we can rely on Model utility functions again !

    # train
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
              validation_data=(x_test, y_test))

    # evaluate on test set
    scores = model.evaluate(x_test, y_test, batch_size, verbose=2)
    print("Test MSE :", scores)

    saver = tfe.Saver(model.variables)
    saver.save('weights/10_02_custom_layers/weights.ckpt')

Train on 404 samples, validate on 102 samples
Epoch 1/24
404/404 [==============================] - 0s 1ms/step - loss: 439.0670 - val_loss: 129.8505
Epoch 2/24
404/404 [==============================] - 0s 93us/step - loss: 96.3968 - val_loss: 92.0870
Epoch 3/24
404/404 [==============================] - 0s 95us/step - loss: 87.9206 - val_loss: 102.1922
Epoch 4/24
404/404 [==============================] - 0s 99us/step - loss: 85.3385 - val_loss: 63.0205
Epoch 5/24
404/404 [==============================] - 0s 101us/step - loss: 48.4697 - val_loss: 39.8241
Epoch 6/24
404/404 [==============================] - 0s 109us/step - loss: 38.2497 - val_loss: 44.6685
Epoch 7/24
404/404 [==============================] - 0s 98us/step - loss: 42.0978 - val_loss: 36.3873
Epoch 8/24
404/404 [==============================] - 0s 101us/step - loss: 34.2104 - val_loss: 30.5881
Epoch 9/24
404/404 [==============================] - 0s 105us/step - loss: 26.8764 - val_loss: 30.7118
Epoch 10/24
404/404 [